In [ ]:
"""entonces es simplemente dado un estado y lo que se hizo en el estado para aumentar 
el score , luego es hacer factor comunt entre los estados y sus outputs para crear una
estructura equivalente.Pero entonces como hace para saber cual output seleccionar en
primer lieu ? c'est aleatoire s'il ne connait pas l'etat actuel ,apres tu pourra faire
la comparaison d'un nouvel etat par rapport a un autre connu en comparant le nombre de 
bits egaux de l'etat actuel"""

In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [5]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
blue_last_state = None
blue_last_action = None
blue_last_reward = None
blue_experiences = []
blue_saved_states = set()  # Track unique states blue has encountered

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    """Spawn new food ensuring no overlap with players."""
    while True:
        x = random.randint(0, WIDTH - FOOD_SIZE)
        y = random.randint(0, HEIGHT // 2 - FOOD_SIZE)
        new_food = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        # Ensure food doesn't overlap players
        if not player1.colliderect(new_food) and not player2.colliderect(new_food):
            return Food(x, y)

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    screen.fill(BLACK)
    
    # Draw players
    pygame.draw.rect(screen, BLUE, player1)
    pygame.draw.rect(screen, RED, player2)
    
    # Draw food
    for food in food_list:
        pygame.draw.rect(screen, GREEN, food.rect)
    
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement():
    global player2
    keys = pygame.key.get_pressed()
    move_speed = 5
    original_pos = player2.copy()

    # Player 2 movement with collision detection
    if keys[pygame.K_UP] and player2.top > 0:
        test_move = player2.move(0, -move_speed)
        if not test_move.colliderect(player1):
            player2.y -= move_speed
        else:
            player2.y = original_pos.y
            
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        test_move = player2.move(0, move_speed)
        if not test_move.colliderect(player1):
            player2.y += move_speed
        else:
            player2.y = original_pos.y
            
    if keys[pygame.K_LEFT] and player2.left > 0:
        test_move = player2.move(-move_speed, 0)
        if not test_move.colliderect(player1):
            player2.x -= move_speed
        else:
            player2.x = original_pos.x
            
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        test_move = player2.move(move_speed, 0)
        if not test_move.colliderect(player1):
            player2.x += move_speed
        else:
            player2.x = original_pos.x

def blue_player_logic(game_matrix):
    global player1_life, player2_life, blue_last_state, blue_last_action, blue_last_reward, blue_saved_states, player1
    actions = ["UP", "DOWN", "LEFT", "RIGHT", "EAT", "ATTACK"]
    
    # Define the size of the surrounding area to focus on
    surrounding_area_size = 3
    x_start = max(0, player1.left // GRID_SIZE - surrounding_area_size // 2)
    x_end = min(MATRIX_WIDTH, player1.right // GRID_SIZE + surrounding_area_size // 2)
    y_start = max(0, player1.top // GRID_SIZE - surrounding_area_size // 2)
    y_end = min(MATRIX_HEIGHT, player1.bottom // GRID_SIZE + surrounding_area_size // 2)

    # Extract the relevant portion of the game matrix to form the state
    blue_last_state = []
    for y in range(y_start, y_end):
        row = game_matrix[y][x_start:x_end]
        blue_last_state.append(row)

    blue_saved_states.add(tuple(map(tuple, blue_last_state)))

    # Calculate reward based on life difference
    reward = player1_life - blue_last_reward if blue_last_reward is not None else 0
    blue_experiences.append((blue_last_state, blue_last_action, reward))
    blue_last_reward = player1_life

    # Choose a random action
    blue_last_action = random.sample(actions, k=random.randint(1, 3))
    
    # Store original position to check for collisions
    original_pos = player1.copy()
    move_speed = 5

    # Perform the actions with collision detection
    if "UP" in blue_last_action and player1.top > 0:
        test_move = player1.move(0, -move_speed)
        if not test_move.colliderect(player2):
            player1.y -= move_speed
        else:
            player1.y = original_pos.y
            
    if "DOWN" in blue_last_action and player1.bottom < HEIGHT // 2:
        test_move = player1.move(0, move_speed)
        if not test_move.colliderect(player2):
            player1.y += move_speed
        else:
            player1.y = original_pos.y
            
    if "LEFT" in blue_last_action and player1.left > 0:
        test_move = player1.move(-move_speed, 0)
        if not test_move.colliderect(player2):
            player1.x -= move_speed
        else:
            player1.x = original_pos.x
            
    if "RIGHT" in blue_last_action and player1.right < WIDTH:
        test_move = player1.move(move_speed, 0)
        if not test_move.colliderect(player2):
            player1.x += move_speed
        else:
            player1.x = original_pos.x

    if "EAT" in blue_last_action:
        for food in food_list[:]:
            if player1.colliderect(food.rect):
                food_list.remove(food)
                player1_life += 1
                food_list.append(spawn_new_food())

    if "ATTACK" in blue_last_action:
        if player1.colliderect(player2):
            player2_life -= 1

def handle_food():
    global food_list, player1_life, player2_life
    
    for food in food_list[:]:
        if player1.colliderect(food.rect):
            food_list.remove(food)
            player1_life += 1
            food_list.append(spawn_new_food())
            
        if player2.colliderect(food.rect):
            food_list.remove(food)
            player2_life += 1
            food_list.append(spawn_new_food())

def display_debug_info():
    """Display the blue player's state, action, and reward in real-time."""
    debug_area = pygame.Surface((WIDTH, 100))
    debug_area.fill((30, 30, 30))
    
    state_text = font.render(f"State: {blue_last_state}", True, WHITE)
    action_text = font.render(f"Action: {blue_last_action}", True, WHITE)
    reward_text = font.render(f"Reward: {blue_last_reward}", True, WHITE)
    saved_states_text = font.render(f"Saved States: {len(blue_saved_states)}", True, WHITE)
    
    debug_area.blit(state_text, (10, 10))
    debug_area.blit(action_text, (10, 40))
    debug_area.blit(reward_text, (10, 70))
    debug_area.blit(saved_states_text, (10, 100))
    screen.blit(debug_area, (0, HEIGHT - 100))

# Game loop
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    game_matrix = update_matrix()
    blue_player_logic(game_matrix)
    handle_food()
    
    draw_game()
    draw_matrix(game_matrix)
    display_debug_info()

    pygame.display.update()
    clock.tick(60)

pygame.quit()

In [1]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 20
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
blue_last_state = None
blue_last_action = None
blue_last_reward = None
blue_experiences = []
blue_saved_states = set()

# Player action states
player1_eating = False
player2_eating = False
player1_attacking = False
player2_attacking = False

# Attack cooldown
player1_attack_cooldown = 0
player2_attack_cooldown = 0
ATTACK_COOLDOWN = 30  # frames (0.5 second at 60 FPS)

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    """Spawn new food ensuring no overlap with players."""
    while True:
        x = random.randint(0, WIDTH - FOOD_SIZE)
        y = random.randint(0, HEIGHT // 2 - FOOD_SIZE)
        new_food = Food(x, y)
        if not any(new_food.rect.colliderect(food.rect) for food in food_list) and \
           not new_food.rect.colliderect(player1) and \
           not new_food.rect.colliderect(player2):
            return new_food

def check_collision_with_food(player_rect):
    """Check if player collides with any food."""
    return any(food.rect.colliderect(player_rect) for food in food_list)

def handle_movement():
    global player2, player2_eating, player2_attacking, player2_attack_cooldown, player1_life, player2_life
    keys = pygame.key.get_pressed()
    move_speed = 5
    original_pos = player2.copy()

    # Update player2 states
    player2_eating = keys[pygame.K_SPACE]
    player2_attacking = keys[pygame.K_LSHIFT]

    # Handle attack cooldown
    if player2_attack_cooldown > 0:
        player2_attack_cooldown -= 1

    # Handle attack with expanded hitbox
    if player2_attacking and player2_attack_cooldown == 0:
        attack_range = 10  # pixels
        attack_hitbox = player2.inflate(attack_range, attack_range)
        if attack_hitbox.colliderect(player1):
            player1_life -= 1
            player2_life += 1
            player2_attack_cooldown = ATTACK_COOLDOWN
            print("Attack hit!")  # Debug message

    # Player 2 movement with collision detection
    if keys[pygame.K_UP] and player2.top > 0:
        test_move = player2.move(0, -move_speed)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.y -= move_speed
        else:
            player2.y = original_pos.y
            
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        test_move = player2.move(0, move_speed)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.y += move_speed
        else:
            player2.y = original_pos.y
            
    if keys[pygame.K_LEFT] and player2.left > 0:
        test_move = player2.move(-move_speed, 0)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.x -= move_speed
        else:
            player2.x = original_pos.x
            
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        test_move = player2.move(move_speed, 0)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.x += move_speed
        else:
            player2.x = original_pos.x

def blue_player_logic(game_matrix):
    global player1_life, player2_life, blue_last_state, blue_last_action, blue_last_reward
    global blue_saved_states, player1, player1_eating, player1_attacking, player1_attack_cooldown
    
    actions = ["UP", "DOWN", "LEFT", "RIGHT", "EAT", "ATTACK"]
    
    # Handle attack cooldown
    if player1_attack_cooldown > 0:
        player1_attack_cooldown -= 1

    # Define the size of the surrounding area to focus on
    surrounding_area_size = 3
    x_start = max(0, player1.left // GRID_SIZE - surrounding_area_size // 2)
    x_end = min(MATRIX_WIDTH, player1.right // GRID_SIZE + surrounding_area_size // 2)
    y_start = max(0, player1.top // GRID_SIZE - surrounding_area_size // 2)
    y_end = min(MATRIX_HEIGHT, player1.bottom // GRID_SIZE + surrounding_area_size // 2)

    # Extract the relevant portion of the game matrix to form the state
    blue_last_state = []
    for y in range(y_start, y_end):
        row = game_matrix[y][x_start:x_end]
        blue_last_state.append(row)

    blue_saved_states.add(tuple(map(tuple, blue_last_state)))

    # Calculate reward based on life difference
    reward = player1_life - blue_last_reward if blue_last_reward is not None else 0
    blue_experiences.append((blue_last_state, blue_last_action, reward))
    blue_last_reward = player1_life

    # Choose a random action
    blue_last_action = random.sample(actions, k=random.randint(1, 3))
    
    # Update player1 states
    player1_eating = "EAT" in blue_last_action
    player1_attacking = "ATTACK" in blue_last_action
    
    # Store original position to check for collisions
    original_pos = player1.copy()
    move_speed = 5

    # Perform the actions with collision detection
    if "UP" in blue_last_action and player1.top > 0:
        test_move = player1.move(0, -move_speed)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.y -= move_speed
        else:
            player1.y = original_pos.y
            
    if "DOWN" in blue_last_action and player1.bottom < HEIGHT // 2:
        test_move = player1.move(0, move_speed)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.y += move_speed
        else:
            player1.y = original_pos.y
            
    if "LEFT" in blue_last_action and player1.left > 0:
        test_move = player1.move(-move_speed, 0)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.x -= move_speed
        else:
            player1.x = original_pos.x
            
    if "RIGHT" in blue_last_action and player1.right < WIDTH:
        test_move = player1.move(move_speed, 0)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.x += move_speed
        else:
            player1.x = original_pos.x

    # Handle attack if selected with expanded hitbox
    if player1_attacking and player1_attack_cooldown == 0:
        attack_range = 10  # pixels
        attack_hitbox = player1.inflate(attack_range, attack_range)
        if attack_hitbox.colliderect(player2):
            player2_life -= 1
            player1_life += 1
            player1_attack_cooldown = ATTACK_COOLDOWN
            print("AI attack hit!")  # Debug message

def handle_food():
    global food_list, player1_life, player2_life
    
    for food in food_list[:]:
        if player1.colliderect(food.rect) and player1_eating:
            food_list.remove(food)
            player1_life += 1
            food_list.append(spawn_new_food())
            
        if player2.colliderect(food.rect) and player2_eating:
            food_list.remove(food)
            player2_life += 1
            food_list.append(spawn_new_food())

def draw_game():
    screen.fill(BLACK)
    
    # Draw players with attack indication
    player1_color = YELLOW if player1_attacking else BLUE
    player2_color = YELLOW if player2_attacking else RED
    pygame.draw.rect(screen, player1_color, player1)
    pygame.draw.rect(screen, player2_color, player2)
    
    # Draw food
    for food in food_list:
        color = BRIGHT_GREEN if player1_eating or player2_eating else GREEN
        pygame.draw.rect(screen, color, food.rect)
    
    # Draw player stats
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    eat_text1 = font.render("Eating" if player1_eating else "Not Eating", True, WHITE)
    eat_text2 = font.render("Eating" if player2_eating else "Not Eating", True, WHITE)
    attack_text1 = font.render("Attack Ready" if player1_attack_cooldown == 0 else f"Cooldown: {player1_attack_cooldown//6}", True, WHITE)
    attack_text2 = font.render("Attack Ready" if player2_attack_cooldown == 0 else f"Cooldown: {player2_attack_cooldown//6}", True, WHITE)
    
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))
    screen.blit(eat_text1, (10, 40))
    screen.blit(eat_text2, (WIDTH - 200, 40))
    screen.blit(attack_text1, (10, 70))
    screen.blit(attack_text2, (WIDTH - 200, 70))

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def display_debug_info():
    debug_area = pygame.Surface((WIDTH, 100))
    debug_area.fill((30, 30, 30))
    
    state_text = font.render(f"State: {blue_last_state}", True, WHITE)
    action_text = font.render(f"Action: {blue_last_action}", True, WHITE)
    reward_text = font.render(f"Reward: {blue_last_reward}", True, WHITE)
    saved_states_text = font.render(f"Saved States: {len(blue_saved_states)}", True, WHITE)
    
    debug_area.blit(state_text, (10, 10))
    debug_area.blit(action_text, (10, 40))
    debug_area.blit(reward_text, (10, 70))
    debug_area.blit(saved_states_text, (10, 100))
    screen.blit(debug_area, (0, HEIGHT - 100))

# Game loop
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    game_matrix = update_matrix()
    blue_player_logic(game_matrix)
    handle_food()
    
    draw_game()
    draw_matrix(game_matrix)
    display_debug_info()

    pygame.display.update()
    clock.tick(60)

pygame.quit()

pygame 2.1.3 (SDL 2.0.22, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
import pygame
import random
import csv
from datetime import datetime

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 20
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
blue_last_state = None
blue_last_action = None
blue_last_reward = None
blue_experiences = []
blue_saved_states = set()

# History tracking
game_moves = []

# Player action states
player1_eating = False
player2_eating = False
player1_attacking = False
player2_attacking = False

# Attack cooldown
player1_attack_cooldown = 0
player2_attack_cooldown = 0
ATTACK_COOLDOWN = 30  # frames (0.5 second at 60 FPS)

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    while True:
        x = random.randint(0, WIDTH - FOOD_SIZE)
        y = random.randint(0, HEIGHT // 2 - FOOD_SIZE)
        new_food = Food(x, y)
        if not any(new_food.rect.colliderect(food.rect) for food in food_list) and \
           not new_food.rect.colliderect(player1) and \
           not new_food.rect.colliderect(player2):
            return new_food

def check_collision_with_food(player_rect):
    return any(food.rect.colliderect(player_rect) for food in food_list)

def handle_movement():
    global player2, player2_eating, player2_attacking, player2_attack_cooldown, player1_life, player2_life
    keys = pygame.key.get_pressed()
    move_speed = 5
    original_pos = player2.copy()

    player2_eating = keys[pygame.K_SPACE]
    player2_attacking = keys[pygame.K_LSHIFT]

    if player2_attack_cooldown > 0:
        player2_attack_cooldown -= 1

    if player2_attacking and player2_attack_cooldown == 0:
        attack_range = 10
        attack_hitbox = player2.inflate(attack_range, attack_range)
        if attack_hitbox.colliderect(player1):
            player1_life -= 1
            player2_life += 1
            player2_attack_cooldown = ATTACK_COOLDOWN

    if keys[pygame.K_UP] and player2.top > 0:
        test_move = player2.move(0, -move_speed)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        test_move = player2.move(0, move_speed)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0:
        test_move = player2.move(-move_speed, 0)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        test_move = player2.move(move_speed, 0)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.x += move_speed

def blue_player_logic(game_matrix):
    global player1_life, player2_life, blue_last_state, blue_last_action, blue_last_reward
    global blue_saved_states, player1, player1_eating, player1_attacking, player1_attack_cooldown
    
    actions = ["UP", "DOWN", "LEFT", "RIGHT", "EAT", "ATTACK"]
    
    if player1_attack_cooldown > 0:
        player1_attack_cooldown -= 1

    surrounding_area_size = 3
    x_start = max(0, player1.left // GRID_SIZE - surrounding_area_size // 2)
    x_end = min(MATRIX_WIDTH, player1.right // GRID_SIZE + surrounding_area_size // 2)
    y_start = max(0, player1.top // GRID_SIZE - surrounding_area_size // 2)
    y_end = min(MATRIX_HEIGHT, player1.bottom // GRID_SIZE + surrounding_area_size // 2)

    blue_last_state = []
    for y in range(y_start, y_end):
        row = game_matrix[y][x_start:x_end]
        blue_last_state.append(row)

    blue_saved_states.add(tuple(map(tuple, blue_last_state)))

    reward = player1_life - blue_last_reward if blue_last_reward is not None else 0
    blue_experiences.append((blue_last_state, blue_last_action, reward))
    blue_last_reward = player1_life

    blue_last_action = random.sample(actions, k=random.randint(1, 3))
    
    player1_eating = "EAT" in blue_last_action
    player1_attacking = "ATTACK" in blue_last_action
    
    original_pos = player1.copy()
    move_speed = 5

    # Record move data
    game_moves.append({
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'state': str(blue_last_state),
        'action': str(blue_last_action),
        'reward': reward,
        'player1_life': player1_life,
        'player2_life': player2_life,
        'player1_position': f"{player1.x},{player1.y}",
        'player2_position': f"{player2.x},{player2.y}"
    })

    if "UP" in blue_last_action and player1.top > 0:
        test_move = player1.move(0, -move_speed)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.y -= move_speed
    if "DOWN" in blue_last_action and player1.bottom < HEIGHT // 2:
        test_move = player1.move(0, move_speed)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.y += move_speed
    if "LEFT" in blue_last_action and player1.left > 0:
        test_move = player1.move(-move_speed, 0)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.x -= move_speed
    if "RIGHT" in blue_last_action and player1.right < WIDTH:
        test_move = player1.move(move_speed, 0)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.x += move_speed

    if player1_attacking and player1_attack_cooldown == 0:
        attack_range = 10
        attack_hitbox = player1.inflate(attack_range, attack_range)
        if attack_hitbox.colliderect(player2):
            player2_life -= 1
            player1_life += 1
            player1_attack_cooldown = ATTACK_COOLDOWN

def handle_food():
    global food_list, player1_life, player2_life
    
    for food in food_list[:]:
        if player1.colliderect(food.rect) and player1_eating:
            food_list.remove(food)
            player1_life += 1
            food_list.append(spawn_new_food())
            
        if player2.colliderect(food.rect) and player2_eating:
            food_list.remove(food)
            player2_life += 1
            food_list.append(spawn_new_food())

def draw_game():
    screen.fill(BLACK)
    
    player1_color = YELLOW if player1_attacking else BLUE
    player2_color = YELLOW if player2_attacking else RED
    pygame.draw.rect(screen, player1_color, player1)
    pygame.draw.rect(screen, player2_color, player2)
    
    for food in food_list:
        color = BRIGHT_GREEN if player1_eating or player2_eating else GREEN
        pygame.draw.rect(screen, color, food.rect)
    
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    eat_text1 = font.render("Eating" if player1_eating else "Not Eating", True, WHITE)
    eat_text2 = font.render("Eating" if player2_eating else "Not Eating", True, WHITE)
    attack_text1 = font.render("Attack Ready" if player1_attack_cooldown == 0 else f"Cooldown: {player1_attack_cooldown//6}", True, WHITE)
    attack_text2 = font.render("Attack Ready" if player2_attack_cooldown == 0 else f"Cooldown: {player2_attack_cooldown//6}", True, WHITE)
    
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))
    screen.blit(eat_text1, (10, 40))
    screen.blit(eat_text2, (WIDTH - 200, 40))
    screen.blit(attack_text1, (10, 70))
    screen.blit(attack_text2, (WIDTH - 200, 70))

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def display_debug_info():
    debug_area = pygame.Surface((WIDTH, 100))
    debug_area.fill((30, 30, 30))
    
    state_text = font.render(f"State: {blue_last_state}", True, WHITE)
    action_text = font.render(f"Action: {blue_last_action}", True, WHITE)
    reward_text = font.render(f"Reward: {blue_last_reward}", True, WHITE)
    saved_states_text = font.render(f"Saved States: {len(blue_saved_states)}", True, WHITE)
    
    debug_area.blit(state_text, (10, 10))
    debug_area.blit(action_text, (10, 40))
    debug_area.blit(reward_text, (10, 70))
    debug_area.blit(saved_states_text, (10, 100))
    screen.blit(debug_area, (0, HEIGHT - 100))

# Game loop
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            # Save game history to CSV
            filename = f'game_history_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            with open(filename, 'w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=['timestamp', 'state', 'action', 'reward', 
                                                     'player1_life', 'player2_life', 
                                                     'player1_position', 'player2_position'])
                writer.writeheader()
                writer.writerows(game_moves)

    handle_movement()
    game_matrix = update_matrix()
    blue_player_logic(game_matrix)
    handle_food()
    
    draw_game()
    draw_matrix(game_matrix)
    display_debug_info()

    pygame.display.update()
    clock.tick(60)

pygame.quit()
#this one produces the file ,ask now to save the unique combination state-command 
#inorder to afterwards search the right one 

pygame 2.1.3 (SDL 2.0.22, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pygame
import random
import csv
import json
from datetime import datetime
import hashlib
from dataclasses import dataclass
from typing import List, Dict

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 800  # Increased height to accommodate visualization
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game with Logic Gate Visualization")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 20
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

@dataclass
class LogicState:
    matrix: List[List[int]]
    commands: List[str]
    timestamp: str

class LogicStateTracker:
    def __init__(self):
        self.states: List[LogicState] = []
        self.load_states()
    
    def add_state(self, matrix, commands):
        if matrix and commands:  # Only add if both exist
            state = LogicState(
                matrix=matrix,
                commands=commands,
                timestamp=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            )
            self.states.append(state)
            self.save_states()
    
    def load_states(self):
        try:
            with open('logic_states.json', 'r') as f:
                data = json.load(f)
                self.states = [LogicState(**state) for state in data]
        except FileNotFoundError:
            self.states = []
    
    def save_states(self):
        with open('logic_states.json', 'w') as f:
            json.dump([state.__dict__ for state in self.states], f, indent=2)

    def draw_visualization(self, screen, start_y):
        CELL_SIZE = 30
        GATE_WIDTH = 60
        GATE_HEIGHT = 40
        
        # Show only the last 3 states to avoid overcrowding
        visible_states = self.states[-3:] if self.states else []
        
        for state_idx, state in enumerate(visible_states):
            y_offset = start_y + (state_idx * 150)
            
            # Draw matrix
            for row_idx, row in enumerate(state.matrix):
                for col_idx, cell in enumerate(row):
                    x = 50 + (col_idx * CELL_SIZE)
                    y = y_offset + (row_idx * CELL_SIZE)
                    color = (0, 100, 200) if cell == 1 else (40, 40, 50)
                    pygame.draw.rect(screen, color, (x, y, CELL_SIZE-2, CELL_SIZE-2))
                    # Draw cell value
                    font = pygame.font.Font(None, 24)
                    text = font.render(str(cell), True, WHITE)
                    screen.blit(text, (x + 10, y + 10))
            
            # Draw AND gate
            gate_x = 250
            gate_y = y_offset + 20
            
            # Draw gate body
            pygame.draw.arc(screen, WHITE, 
                          (gate_x, gate_y, GATE_WIDTH, GATE_HEIGHT),
                          0, 3.14159, 2)
            pygame.draw.line(screen, WHITE,
                           (gate_x, gate_y),
                           (gate_x, gate_y + GATE_HEIGHT), 2)
            
            # Connect 1s to gate inputs
            ones_positions = [(row_idx, col_idx) 
                            for row_idx, row in enumerate(state.matrix)
                            for col_idx, cell in enumerate(row) if cell == 1]
            
            for idx, (row, col) in enumerate(ones_positions):
                start_x = 50 + (col * CELL_SIZE) + CELL_SIZE//2
                start_y = y_offset + (row * CELL_SIZE) + CELL_SIZE//2
                end_x = gate_x
                end_y = gate_y + (idx * 10)
                pygame.draw.line(screen, WHITE,
                               (start_x, start_y), (end_x, end_y), 2)
            
            # Draw commands and connect to gate output
            for cmd_idx, cmd in enumerate(state.commands):
                cmd_x = gate_x + GATE_WIDTH + 50
                cmd_y = y_offset + (cmd_idx * 30)
                
                # Command box
                pygame.draw.rect(screen, (60, 60, 70),
                               (cmd_x, cmd_y, 80, 25))
                
                # Command text
                text = font.render(cmd, True, WHITE)
                screen.blit(text, (cmd_x + 10, cmd_y + 5))
                
                # Connect gate to command
                pygame.draw.line(screen, WHITE,
                               (gate_x + GATE_WIDTH, gate_y + GATE_HEIGHT//2),
                               (cmd_x, cmd_y + 12), 2)

# Food class
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)

# Initialize game objects
food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
blue_last_state = None
blue_last_action = None
blue_last_reward = None

# Attack cooldown
player1_attack_cooldown = 0
player2_attack_cooldown = 0
ATTACK_COOLDOWN = 30

# Player action states
player1_eating = False
player2_eating = False
player1_attacking = False
player2_attacking = False

# Initialize the logic tracker
logic_tracker = LogicStateTracker()

def spawn_new_food():
    while True:
        x = random.randint(0, WIDTH - FOOD_SIZE)
        y = random.randint(0, HEIGHT // 2 - FOOD_SIZE)
        new_food = Food(x, y)
        if not any(new_food.rect.colliderect(food.rect) for food in food_list) and \
           not new_food.rect.colliderect(player1) and \
           not new_food.rect.colliderect(player2):
            return new_food

def check_collision_with_food(player_rect):
    return any(food.rect.colliderect(player_rect) for food in food_list)

def handle_movement():
    global player2, player2_eating, player2_attacking, player2_attack_cooldown, player1_life, player2_life
    keys = pygame.key.get_pressed()
    move_speed = 5

    player2_eating = keys[pygame.K_SPACE]
    player2_attacking = keys[pygame.K_LSHIFT]

    if player2_attack_cooldown > 0:
        player2_attack_cooldown -= 1

    if player2_attacking and player2_attack_cooldown == 0:
        attack_range = 10
        attack_hitbox = player2.inflate(attack_range, attack_range)
        if attack_hitbox.colliderect(player1):
            player1_life -= 1
            player2_life += 1
            player2_attack_cooldown = ATTACK_COOLDOWN

    if keys[pygame.K_UP] and player2.top > 0:
        test_move = player2.move(0, -move_speed)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        test_move = player2.move(0, move_speed)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0:
        test_move = player2.move(-move_speed, 0)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        test_move = player2.move(move_speed, 0)
        if not test_move.colliderect(player1) and (player2_eating or not check_collision_with_food(test_move)):
            player2.x += move_speed

def blue_player_logic(game_matrix):
    global player1_life, player2_life, blue_last_state, blue_last_action, blue_last_reward
    global player1, player1_eating, player1_attacking, player1_attack_cooldown
    
    if player1_attack_cooldown > 0:
        player1_attack_cooldown -= 1

    surrounding_area_size = 3
    x_start = max(0, player1.left // GRID_SIZE - surrounding_area_size // 2)
    x_end = min(MATRIX_WIDTH, player1.right // GRID_SIZE + surrounding_area_size // 2)
    y_start = max(0, player1.top // GRID_SIZE - surrounding_area_size // 2)
    y_end = min(MATRIX_HEIGHT, player1.bottom // GRID_SIZE + surrounding_area_size // 2)

    current_state = []
    for y in range(y_start, y_end):
        row = game_matrix[y][x_start:x_end]
        current_state.append(row)

    actions = ["UP", "DOWN", "LEFT", "RIGHT", "EAT", "ATTACK"]
    blue_last_action = random.sample(actions, k=random.randint(1, 3))
    blue_last_state = current_state

    # Add state to logic tracker
    logic_tracker.add_state(current_state, blue_last_action)

    player1_eating = "EAT" in blue_last_action
    player1_attacking = "ATTACK" in blue_last_action
    
    move_speed = 5

    if "UP" in blue_last_action and player1.top > 0:
        test_move = player1.move(0, -move_speed)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.y -= move_speed
    if "DOWN" in blue_last_action and player1.bottom < HEIGHT // 2:
        test_move = player1.move(0, move_speed)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.y += move_speed
    if "LEFT" in blue_last_action and player1.left > 0:
        test_move = player1.move(-move_speed, 0)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.x -= move_speed
    if "RIGHT" in blue_last_action and player1.right < WIDTH:
        test_move = player1.move(move_speed, 0)
        if not test_move.colliderect(player2) and (player1_eating or not check_collision_with_food(test_move)):
            player1.x += move_speed

    if player1_attacking and player1_attack_cooldown == 0:
        attack_range = 10
        attack_hitbox = player1.inflate(attack_range, attack_range)
        if attack_hitbox.colliderect(player2):
            player2_life -= 1
            player1_life += 1
            player1_attack_cooldown = ATTACK_COOLDOWN

def handle_food():
    global food_list, player1_life, player2_life
    
    for food in food_list[:]:
        if player1.colliderect(food.rect) and player1_eating:
            food_list.remove(food)
            player1_life += 1
            food_list.append(spawn_new_food())
            
        if player2.colliderect(food.rect) and player2_eating:
            food_list.remove(food)
            player2_life += 1
            food_list.append(spawn_new_food())

def draw_game():
    screen.fill(BLACK)
    
    # Draw game area (top half)
    pygame.draw.rect(screen, (20, 20, 30), (0, 0, WIDTH, HEIGHT // 2))
    
    player1_color = YELLOW if player1_attacking else BLUE
    player2_color = YELLOW if player2_attacking else RED
    pygame.draw.rect(screen, player1_color, player1)
    pygame.draw.rect(screen, player2_color, player2)
    
    for food in food_list:
        color = BRIGHT_GREEN if player1_eating or player2_eating else GREEN
        pygame.draw.rect(screen, color, food.rect)
    
    font = pygame.font.Font(None, 36)
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    eat_text1 = font.render("Eating" if player1_eating else "Not Eating", True, WHITE)
    eat_text2 = font.render("Eating" if player2_eating else "Not Eating", True, WHITE)
    attack_text1 = font.render("Attack Ready" if player1_attack_cooldown == 0 else f"Cooldown: {player1_attack_cooldown//6}", True, WHITE)
    attack_text2 = font.render("Attack Ready" if player2_attack_cooldown == 0 else f"Cooldown: {player2_attack_cooldown//6}", True, WHITE)
    
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))
    screen.blit(eat_text1, (10, 40))
    screen.blit(eat_text2, (WIDTH - 200, 40))
    screen.blit(attack_text1, (10, 70))
    screen.blit(attack_text2, (WIDTH - 200, 70))

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.